We keep the same model but replace the random sampler by imbalance sampler

In [1]:
%pip install torchsampler

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0,),std=(1,))])        
}
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_data_size = len(train_data_augmented)
test_data_size = len(test_data)

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from torchsampler import ImbalancedDatasetSampler

batch_size = 32

train_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
valid_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)
classes = ('noface','face')

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

In [6]:
def train_NN(n_epochs, optimizer, model, criterion, early_stop_threshold):
    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # print statistics after every epoch
        loss_average = running_loss / len(train_loader)
        print(f'[{epoch}, {i + 1:5d}] loss: {loss_average:.3f}')
        
        # stop the training if the loss is lower than the threshold
        if (loss_average < early_stop_threshold):
            break


    print('Finished Training')

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20
early_stop_threshold = 0.02

train_NN(n_epochs, optimizer_SGD, net, criterion, early_stop_threshold)

[1,  2867] loss: 0.663
[2,  2867] loss: 0.494
[3,  2867] loss: 0.404
[4,  2867] loss: 0.335
[5,  2867] loss: 0.293
[6,  2867] loss: 0.263
[7,  2867] loss: 0.237
[8,  2867] loss: 0.210
[9,  2867] loss: 0.191
[10,  2867] loss: 0.174
[11,  2867] loss: 0.162
[12,  2867] loss: 0.154
[13,  2867] loss: 0.144
[14,  2867] loss: 0.140
[15,  2867] loss: 0.131
[16,  2867] loss: 0.130
[17,  2867] loss: 0.127
[18,  2867] loss: 0.126
[19,  2867] loss: 0.124
[20,  2867] loss: 0.119
Finished Training


In [8]:
PATH = './3-layer-imbalance-sampler-net.pth'
torch.save(net.state_dict(), PATH)

In [9]:
def get_classifier_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [10]:
model_accuracy = get_classifier_accuracy(net)
print('Accuracy of the 3-layer no augmentation classifier on the {} test images: {:.2f}'.format(test_data_size, model_accuracy))

Accuracy of the 3-layer no augmentation classifier on the 7628 test images: 97.31
